In [17]:
import os
import ee
import asf_search
import pandas as pd
import geopandas as gpd
from hyp3_sdk import HyP3

In [14]:
# Get the current working directory
current_directory = os.getcwd()

# Get the parent directory (one level up)
parent_directory = os.path.dirname(current_directory)

data_dir = os.path.join(parent_directory, 'data')

In [33]:
## fire perimeters

fnames = os.listdir(os.path.join(data_dir, 'bldg_destruction_pts_sample_fires'))

shps = [f for f in fnames if '.shp' in f and 'xml' not in f]
shp_paths = [os.path.join(data_dir, 'bldg_destruction_pts_sample_fires', f) for f in shps]

fire_gdfs = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in shp_paths]))

In [36]:
fire_gdfs.crs

<Projected CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84" ...>
Name: Albers_Conical_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich